In [156]:
import pandas as pd
import numpy as np
import requests as request #API Response
import json

In [157]:
# Read Criminal Data
# Website using
dataset = pd.read_csv("estimated_crimes_1979_2020.csv")
print(dataset)
# 특정행 [NaN] 일때는 드랍 시킨다
dataset = dataset.dropna(subset=['state_abbr']) 

      year state_abbr          state_name  population  violent_crime  \
0     1979        NaN                 NaN   220099000        1208030   
1     1979         AK              Alaska      406000           1994   
2     1979         AL             Alabama     3769000          15578   
3     1979         AR            Arkansas     2180000           7984   
4     1979         AZ             Arizona     2450000          14528   
...    ...        ...                 ...         ...            ...   
2179  2020         WA          Washington     7693612          22596   
2180  2020         WI           Wisconsin     5832655          18861   
2181  2020         WV       West Virginia     1784787           6352   
2182  2020         WY             Wyoming      582328           1364   
2183  2020        NaN                 NaN   329484123        1313105   

      homicide  rape_legacy  rape_revised  robbery  aggravated_assault  \
0        21460      76390.0           NaN   480700           

In [159]:
# 2000년대 이상, dataframe
after2000year = dataset[dataset['year'] >= 2000]

# 특정행 선택.
after2000yearDF = after2000year[['year', 'violent_crime']]
# print(after2000yearDF)
df = pd.DataFrame(index=range(0,20), columns = {'year','totalCrime'})

yearList = []
# 특정조건들의 합
for n in range(2000, 2021):
    selectedYear = after2000yearDF[after2000year['year'] == n]
    sumYear = selectedYear.sum(axis=0)
    yearList.append({'year': n, 'totalCriminalCount': sumYear['violent_crime']})

# expected Dataset
targetFrame = pd.DataFrame(yearList)
targetFrame

,year,totalCriminalCount
0,2000,1425486
1,2001,1439480
2,2002,1423677
3,2003,1383676
4,2004,1360088
5,2005,1390745
6,2006,1435123
7,2007,1422970
8,2008,1394461
9,2009,1325896


In [161]:
#API request
apiAddress = "https://api.themoviedb.org/3/discover/movie?api_key=e4216107f76e07c77cf69da607dc51a7&language=en-US&sort_by=release_date.desc&include_adult=false&include_video=false&page=1&release_date.gte=2000-01-01&release_date.lte=2001-12-31&with_genres=28&with_original_language=en&with_watch_monetization_types=flatrate"
data = request.get(apiAddress)
dataJson = data.json()
# json value access
dataJson['total_results']
dataJson

{'page': 1,
 'results': [{'adult': False,
   'backdrop_path': '/xd6A8LAPJLZ6qHm8uLGVh8PJ3Dn.jpg',
   'genre_ids': [27, 28, 53, 878],
   'id': 72231,
   'original_language': 'en',
   'original_title': 'Flying Virus',
   'overview': "After a series of Amazonian Indian attacks on US owned petroleum installation in Brazil, both governments start a secret 'special program'. In fact colonel Ezekial's men use GM killer bees to eradicate the tribes. During an Indian attack, reporter Ann Bauer is stung, yet survives after a mysterious rescue. Dr. Stephen North realizes the venom has priceless healing powers and smuggles the bee aboard a flight to New York. Bauer, her nearly-divorced husband, Martin, who is aboard and her friend, US State Department project representative, Scotty, face potential mass-killing after turbulence releases the bees aboard.",
   'popularity': 3.855,
   'poster_path': '/xDEJVQMBRPiXhL8QEC2gD3asxDx.jpg',
   'release_date': '2001-12-31',
   'title': 'Flying Virus',
   'vi

In [164]:
movieList = []
for year in range(2000,2021):
    apiAddress = "https://api.themoviedb.org/3/discover/movie?api_key=e4216107f76e07c77cf69da607dc51a7&language=en-US&sort_by=release_date.desc&include_adult=false&include_video=false&page=1&release_date.gte={}-01-01&release_date.lte={}-12-31&with_genres=28&with_original_language=en&with_watch_monetization_types=flatrate".format(year, year)
    data = request.get(apiAddress)
    jsonData = data.json()
    movieTotalCount = jsonData['total_results']
    movieList.append(movieTotalCount)
    
# movie totalCount Added
print(movieList)

[365, 437, 551, 525, 448, 497, 500, 544, 631, 633, 645, 685, 680, 697, 697, 706, 778, 715, 755, 691, 709]


In [165]:
print(len(movieList))

21


In [166]:
targetFrame['movieCount'] = movieList

In [167]:
targetFrame

,year,totalCriminalCount,movieCount
0,2000,1425486,365
1,2001,1439480,437
2,2002,1423677,551
3,2003,1383676,525
4,2004,1360088,448
5,2005,1390745,497
6,2006,1435123,500
7,2007,1422970,544
8,2008,1394461,631
9,2009,1325896,633


In [168]:
# Calcualte correlation between two columns 
corr = targetFrame['totalCriminalCount'].corr(targetFrame['movieCount'])
print("Correlation between totalCriminalCount and movieCount is: " + str(corr))
corr = targetFrame['year'].corr(targetFrame['totalCriminalCount'])
print("Correlation between year and totalCriminalCount is: " + str(corr))


Correlation between totalCriminalCount and movieCount is: -0.8009708446869702
Correlation between year and totalCriminalCount is: -0.753558919281101


In [138]:
# 2000년대 이후, 인구가 폭력사건에 영향이 있는지 알아보기
after2000year = dataset[dataset['year'] >= 2000]

# 특정행 선택.
targetDF = after2000year[['year','population','violent_crime']]
corr = targetDF['population'].corr(targetDF['violent_crime'])
print("Correlation between population and violent_crime is: " + str(corr))

Correlation between population and violent_crime is: 0.9592357246722923


In [154]:
# movie after 2000 year
dataset = pd.read_csv("estimated_crimes_1979_2020.csv")
movieAfter2000year = dataset[dataset['year'] >= 2000]
movieAfter2000year = movieAfter2000year[['year', 'violent_crime', 'state_name']]
movieAfter2000year

,year,violent_crime,state_name
1092,2000,1425486,NaN
1093,2000,3554,Alaska
1094,2000,21620,Alabama
1095,2000,11904,Arkansas
1096,2000,27281,Arizona
...,...,...,...
2179,2020,22596,Washington
2180,2020,18861,Wisconsin
2181,2020,6352,West Virginia
2182,2020,1364,Wyoming


In [155]:
electionDataSet = pd.read_csv("1976-2020-senate.csv", engine='python')
electionAfter2000 = electionDataSet[electionDataSet['year'] >= 2000]
electionAfter2000

,year,state,state_po,state_fips,state_cen,state_ic,office,district,stage,special,candidate,party_detailed,writein,mode,candidatevotes,totalvotes,unofficial,version,party_simplified
1631,2000,ARIZONA,AZ,4,86,61,US SENATE,statewide,gen,False,WILLIAM TOEL,INDEPENDENT,False,total,109230,1397076,False,20210114,OTHER
1632,2000,ARIZONA,AZ,4,86,61,US SENATE,statewide,gen,False,VANCE HANSEN,GREEN,False,total,108926,1397076,False,20210114,OTHER
1633,2000,ARIZONA,AZ,4,86,61,US SENATE,statewide,gen,False,JON KYL,REPUBLICAN,False,total,1108196,1397076,False,20210114,REPUBLICAN
1634,2000,ARIZONA,AZ,4,86,61,US SENATE,statewide,gen,False,BARRY J. HESS II,LIBERTARIAN,False,total,70724,1397076,False,20210114,LIBERTARIAN
1635,2000,CALIFORNIA,CA,6,93,71,US SENATE,statewide,gen,False,"JOSE LUIS ""JOE"" CAMAHORT",REFORM,False,total,96552,10623608,False,20210114,OTHER
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3624,2020,WYOMING,WY,56,83,68,US SENATE,statewide,gen,False,UNDER VOTES,NaN,False,total,6401,278503,False,20210114,OTHER
3625,2021,GEORGIA,GA,13,58,44,US SENATE,statewide,runoff,True,KELLY LOEFFLER,REPUBLICAN,False,total,2194848,4483294,True,20210114,REPUBLICAN
3626,2021,GEORGIA,GA,13,58,44,US SENATE,statewide,runoff,True,RAPHAEL WARNOCK,DEMOCRAT,False,total,2288446,4483294,True,20210114,DEMOCRAT
3627,2021,GEORGIA,GA,13,58,44,US SENATE,statewide,runoff,False,DAVID A. PERDUE,REPUBLICAN,False,total,2213979,4483241,True,20210114,REPUBLICAN
